<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2016 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
</p>

# _Regular Expressions_

Hasta ahora cada vez que necesitamos procesar la información contenida en _strings_ lo hemos hecho mediante los mismos métodos que los objetos de esta clase (`str`) nos provee. Esto es fácil cuando el texto que analizamos es relativamente simple. Por ejemplo si queremos separar un _string_ de acuerdo a un carácter especial solo deberíamos hacer `string.split('caracter')`; o bien si necesitamos buscar una secuencia dentro de un string cualquiera podemos usar el método `find()`. Sin embargo, cuando la información está contenida en _strings_ complejos o bien cuando necesitamos buscar múltiples secuencias dentro de información, existe una forma más eficiente y poderosa usando patrones.

Estos patrones se conocen como **expresiones regulares** o **RE** o **regex**. Las expresiones regulares corresponden a secuencias especiales de caracteres que nos permiten comparar, buscar _strings_ o conjuntos de _strings_, entre otras. Las expresiones regulares se definen como un patrón mediante una sintaxis especializada. Algunos ejemplos de situaciones que pueden ser resueltas usando expresiones son: validación de formularios, búsqueda y reemplazo, transformación de texto, y procesamiento de registros (_logs_).

Aunque hoy en día varios lenguajes permiten el uso de expresiones regulares, estas vienen siendo usadas desde el año 1956 cuando fueron propuestas por el matemático [Stephen Kleene](https://en.wikipedia.org/wiki/Stephen_Cole_Kleene). 

Las expresiones regulares se deben definir utilizando un conjunto de caracteres denominados **meta-carateres**. Algunos de ellos son:

- **```[ ]```**: usados para especificar clases de caracteres. Los meta-caracteres no funcionan dentro de las clases. 

- **```-```**: permite definir rangos de caracteres. Se usan en conjunto con "`[ ]`".

- **```+```**: usado para indicar que la clase definida estará presente una o más veces.

- **```*```**: cumple la función como complemento de "`+`", *i.e.*, que la expresión definida en una clase no aparezca ninguna vez en el string.

- **```?```**: permite indicar que la expresión definida en el patrón puede estar una vez, o no estar.

- **```.```**: usado para representar cualquier carácter, excepto un salto de línea.

- **```( )```**: cumplen la función de delimitadores

- **```|```**: utilizado para separar grupos de secuencias. Con esto se considerará que en ese lugar puedan ser consideradas los grupos de patrones.

- **```\```**: permite que los meta-caracteres puedan ser incorporados como parte del patrón y no sean considerados como especiales
- **```{m, n}```**: usado para indicar que la clase definida estará presente entre m y n. También puede ser una cantidad fija m.

Python provee el módulo `re` para el uso de expresiones regulares. Dentro de las funcionalidades que nos permite el módulo `re` de Python se encuentran:

- Verificar si un string cumple con un patrón: ``re.match()``
- Revisar si algún sub-string cumple con el patrón: ``re.search()``
- Realizar el reemplazo del patrón por otra secuencia de caracteres: ``re.sub()``
- Separar un string de acuerdo al patrón: ``re.split()``



### _Matching_

Las expresiones regulares son fuertemente utilizadas en operaciones de comparación o _matching_. A continuación revisaremos algunos ejemplos y sus resultados.

In [2]:
# Para cargar el módulo re
import re

In [3]:
# Definamos un conjunto de secuencias que necesitamos verificar cumplen con 
# un determinado patrón.
seq = ["4tt", "4ttkbcabc32", "3ssafjabc33", "4tssssghj33", "44ttabcdag60", "4ttabcfgh41", "3ttabc4ttyb"]
for s in seq:
    print(re.match('(4tt.b)', s))

None
<_sre.SRE_Match object; span=(0, 5), match='4ttkb'>
None
None
None
<_sre.SRE_Match object; span=(0, 5), match='4ttab'>
None


El patrón que estamos utilizando es `^(4tt.b)`. El meta-caracter `^` nos permite indicar que la subsecuencia `(4tt.b)` **debe** estar en la primera casilla. La delimitamos como grupo usando "`( )`". A pesar del patrón, el método ```match()``` de **re** siempre busca al comienzo del string.

El método match retorna un objeto tipo `Match`, donde algunos de los atributos más relevantes son:

- `span`: tupla que indica el inicio y término del patrón comparado

- `group(índice)`: el patrón que fue comparado y que aparece en el campo `match`.


El resultado de `match()` puede ser utilizado directamente como condición de sentencias `if`, `while`, etc. En el siguiente ejemplo verificaremos si las secuencias en `seq` cumplen con el patrón de tener la sub-secuencia caracteres `4tt.b` al comienzo de la secuencia. 


In [9]:
# Recordar las secuencias
# seq = ["4tt", "4ttabcabc32", "3ssafjabc33", "4tssssghj33", "44ttabcdag60", "4ttabcfgh41", "3ttabc4tt"]

for s in seq:    
    if re.match('^(4tt)', s):
        print("{} cumple con el patron".format(s))

4tt cumple con el patron
4ttkbcabc32 cumple con el patron
4ttabcfgh41 cumple con el patron


Ahora modifiquemos el patrón para que permita detectar en repeticiones de un grupo de caracteres. Para esto incorporaremos al  patrón utilizado el grupo de caracteres como `(abc)`.

In [15]:
for s in seq:
    if re.match('^4tt(abc){1}', s):
        print("{} cumple con el patron".format(s))

4ttabcfgh41 cumple con el patron


In [16]:
for s in seq:
    # Indicaremos con los '{ }' el número de veces que el grupo debe estar presente.
    # Como vemos, por defecto se asume que puede estar 1 o más veces.
    if re.match('^4tt(abc){0}', s):
        print("{} cumple con el patron".format(s))

4tt cumple con el patron
4ttkbcabc32 cumple con el patron
4ttabcfgh41 cumple con el patron


Veamos ahora como podríamos asegurarnos de que una dirección de correo electrónico cumpla con un determinado formato. Por ejemplo, supongamos que las direcciones que admitiremos pueden tener cualquier tipo de caracter antes de la `@` y pertenecer a al los dominios `mail.cl` o `mimail.cl`, y con esto a cualquier de las direcciones que incluyan este dominio, como por ejemplo: `seccion1.mimail.cl`, `seccion2.mimail.cl`. Para construir el patrón que nos permitirá verficar esta información, debemos hacerlo de la siguiente forma:

1. Al comienzo debemos incluir los meta-caracteres `[a-zA-Z0-9_.]+`. Con esto estamos indicando que la cadena que vamos a ingresar contendrá 1 o más (`+`) caracteres entre letras mayúsculas y minúsculas, números o bien los caracteres "`_`" o "`.`", especificados en la clase mediante "`[ ]`";
1. Luego irá el símbolo "`@`";
1. A continuación debemos indicar que podría o no existir ("`?`") los subdominios `((seccion1|seccion2)\.)?`, seguidos de un "`.`". En este caso debemos usar el meta-caracter "`\`" para hacer que el meta-caracter "`.`" sea considerado como punto.
1. Finalmente, debemos verifcar que esté presente el dominio del correo incluyendo en el patrón `(mi)?mail.cl`, indicando que la secuencia "`mi`" puede estar o no ("`?`") en el dominio. Otra forma de escribir la misma regla es creando el grupo `(mimail|mail).cl`.

In [18]:
def is_valid_mail(email):
    # Recordar que el método 'match()' retorna un objeto de tipo 'Match' que al ser
    # usado en sentencias como IF y WHILE representa un valor lógico. Podemos hacer
    # que una función retorne un valor lógico de la operación de match haciendo la
    # conversión a bool.
    #
    # Otra forma de escribir el patrón es:
    # pattern = "[a-zA-Z0-9_.]+@((seccion1|seccion2)\.)?(mi)?mail.cl"
    pattern = "[a-zA-Z0-9_.]+@((seccion1|seccion2)\.)?(mimail|mail).cl"
    return bool(re.match(pattern, email))

# Las direcciones de correo tienen consistencia con el patrón utilizado
print(is_valid_mail('nombre.apellido@mail.cl'))
print(is_valid_mail('nombre_aprellido@mimail.cl'))
print(is_valid_mail('nombre1010@seccion2.mimail.cl'))

# Estos los correos incluyen elementos no considerados en el patrón
print(is_valid_mail('nombre1010@tumail.cl'))
print(is_valid_mail('nombre101-@tumail.cl'))

True
True
True
False
False


Veamos otro ejemplo que consiste en verificar que el RUT ingresado en un campo de un formulario tenga el formato especificado como: ##.###.###-#, que es una secuencias de números separados por puntos y el dígito verificador separado por guión. Debe considerar que existen números infereriores al millón, pero no menos que mil; y que el digito verificador puede ser un decimal hasta el 9 o la letra "k". Esto es solo a modo de ejemplo. La estructura del patrón entonces quedaría definida de la siguiente forma:

1. Al comienzo, incluiremos la regla que indica que puede existir o no ("`?`") entre uno o dos caracteres numéricos seguidos de un "`.`". Esto lo indicamos como  `[0-9]{1,2}\.?`. Dentro de "`[]`" estamos incluyendo la clase de caracteres numéricos y en "`{1,2}`" estamos indicando que habrá entre uno y dos caracteres en esta sección antes del punto;
1. Después, incluímos los siguientes tres caracteres numéricos. Lo indicamos como `[0-9]{1,3}`;
1. Luego, indicamos que debe haber obligatoriamente 3 dígitos incluyendo seguidos de un guión `[0-9]{3}-`;
1. Finalmente, consideramos que haya un dígito entre 0 y 9, o bien la letra k mayúscula o minúscula.

In [26]:
def is_valid_rut(rut):
    pattern = "([0-9]{1,2}\.[0-9]{3}|[0-9]{1,3})\.[0-9]{3}-([0-9]|k|K)"
    return bool(re.match(pattern, rut))

# Casos válidos para el patrón
print(is_valid_rut('12.224.877-2'))
print(is_valid_rut('12.745.331-k'))
print(is_valid_rut('113.221-2'))

# Casos no válidos para el patrón
print(is_valid_rut('13.427.974-a'))
print(is_valid_rut('ab.111.444-0'))

True
True
True
False
False


Existen abreviaciones prestablecidas para ciertas clases de caracteres. Por ejemplo:

- `\d`: equivale a [0-9];
- `\D`: es equivalente a [^0-9], donde se compara con cualquier caracter que no sea dígito; 
- `\s`: equivale a hacer `[\t\n\r\f\v]`, compara cualquier tipo de espacio en blanco;
- `\S`: equivale a escribir la clase `[^\t\n\r\f\v]`, que compara con cualquier caracter distinto a los espacios en blanco;
- `\w`: es equivalente a la clase [a-zA-Z0-9\_], donde se compara con cualquier caracter alfa numérico;
- `\W`: equivale a [^a-zA-Z0-9\_], que contempla que no haya ningún caracter alfa numérico.

De esta forma podríamos escribir el patrón para comprobar el RUT de la siguiente forma equivalente:

In [27]:
def is_valid_rut(rut):
    pattern = "(\d{1,2}\.)?\d{1,3}\.\d{3}-(\d|k|K)"
    return bool(re.match(pattern, rut))

# Casos válidos para el patrón
print(is_valid_rut('12.224.877-k'))
print(is_valid_rut('ab.111.444-0'))

True
False


### Búsqueda

La búsqueda es otra de las tareas donde comúnmente se aplica el uso de expresiones regulares. En este caso el módulo `re` permite tres formas de búsqueda:

- `search()`: busca en una secuencia cualquier posición donde el patrón coincida, retorna la primera coincidencia.
- `findall()`: Encuentra todas las sub-secuencias donde el patrón coincida, y las retorna como una lista.
- `finditer()`: opera como `findall()`, pero retorna un iterador.

Volvamos al ejemplo de las secuencias usadas anteriormente y busquemos en ellas donde aparece la subsecuencia "ab":

In [36]:
seq = ["4tt", "4ttabcabc32", "3ssafjabc33", "4tssssghj33", "4ttabcdag60", "4ttabcfgh41", "3ttabc4tt"]

for s in seq:
    print('secuencia {}: {}'.format(s, re.search('(ab)', s)))

secuencia 4tt: None
secuencia 4ttabcabc32: <_sre.SRE_Match object; span=(3, 5), match='ab'>
secuencia 3ssafjabc33: <_sre.SRE_Match object; span=(6, 8), match='ab'>
secuencia 4tssssghj33: None
secuencia 4ttabcdag60: <_sre.SRE_Match object; span=(3, 5), match='ab'>
secuencia 4ttabcfgh41: <_sre.SRE_Match object; span=(3, 5), match='ab'>
secuencia 3ttabc4tt: <_sre.SRE_Match object; span=(3, 5), match='ab'>


Al igual que el método `match()`, el método `search()` retorna un objeto indicando la posición de la coincidencia. Si no encuentra alguna coincidencia retorna `None`.

Veamos un ejemplo donde podamos recuperar un listado con todos las subsecuencias de valores numéricos que estén en las secuencias:

In [10]:
for s in seq:
    print('secuencia {}: {}'.format(s, re.findall('\d+', s)))

secuencia 4tt: ['4']
secuencia 4ttabcabc32: ['4', '32']
secuencia 3ssafjabc33: ['3', '33']
secuencia 4tssssghj33: ['4', '33']
secuencia 4ttabcdag60: ['4', '60']
secuencia 4ttabcfgh41: ['4', '41']
secuencia 3ttabc4tt: ['3', '4']


### Sustitución

La modificación de secuencias es también otra de las tareas en que las expresiones regulares son de gran ayuda. El módulo `re` nos provee con el método `sub(<patron>, <reemplazar por>, secuencia)` que nos permite hacer sustitución de acuerdo al patrón indicado. Por ejemplo, eliminaremos todos los números en las secuencias usadas en los ejemplos anteriores:

In [38]:
for s in seq:
    # sub retorna un nuevo valor, por lo tanto, no modifica la secuencia original
    print('Secuencia {} queda como {}'.format(s, re.sub('\d', '', s)))

Secuencia 4tt queda como tt
Secuencia 4ttabcabc32 queda como ttabcabc
Secuencia 3ssafjabc33 queda como ssafjabc
Secuencia 4tssssghj33 queda como tssssghj
Secuencia 4ttabcdag60 queda como ttabcdag
Secuencia 4ttabcfgh41 queda como ttabcfgh
Secuencia 3ttabc4tt queda como ttabctt


En vez de usar un valor a sustituir, podemos también incluir una función donde podamos utilizar una regla de sustitución más compleja. Por ejemplo, procesemos una secuencia de ADN con bases "A", "T" y "C" reemplazándolas por sus bases complementarias:

In [41]:
def bases(base):
    # A la función entra un objeto tipo Match. Debemos recuperar el 
    # valor de la coincidencia haciendo group(0) ya que sabemos que 
    # con el patrón utilizado nos llegará solo una coincidencia.
    mapping = {'A': 'T', 'G': 'C', 'T': 'A', 'C':'G'}
    return mapping[base.group(0)]

adn = 'ACAAGATGCCATTGTCCCCCGGCCTCCTGCTGCTGCTGCTCTCCGGGGCCACGGCCACCG'
print(adn)
print(re.sub('[ATCG]', bases, adn))

ACAAGATGCCATTGTCCCCCGGCCTCCTGCTGCTGCTGCTCTCCGGGGCCACGGCCACCG
TGTTCTACGGTAACAGGGGGCCGGAGGACGACGACGACGAGAGGCCCCGGTGCCGGTGGC


### Split

Por ejemplo, para separar una secuencia por el caracter _e_ podemos usar el método `split()` de los objetos `str`. Esto quedaría como siguiente ejemplo:

In [54]:
msg = "Este es un mensaje simple que vamos a procesar ehh!"
print(msg.split('e'))

['Est', ' ', 's un m', 'nsaj', ' simpl', ' qu', ' vamos a proc', 'sar ', 'hh!']


Podemos realizar el mismo procesamiento usado el módulo `re` de la siguiente forma:

In [55]:
msg = "Este es un mensaje simple que vamos a procesar ehh!"
re.split('[e|E]', msg)

['',
 'st',
 ' ',
 's un m',
 'nsaj',
 ' simpl',
 ' qu',
 ' vamos a proc',
 'sar ',
 'hh!']

El método `split()`require como argumentos un string con el patrón y el string donde vamos aplicar ese patrón. Hasta ahora ambos métodos nos entregan el mismo resultado. Para casos simples de procesamiento solo bastaría el uso de los métodos propios de la clase `str()` y dejar regex para tareas más complejas.

In [15]:
# Divimos sacando solo las vocales. Los patrones regules son case
print(re.split('[aeiou]', msg))

['Est', ' ', 's ', 'n m', 'ns', 'j', ' s', 'mpl', ' q', '', ' v', 'm', 's ', ' pr', 'c', 's', 'r ', 'hh!']


Volvamos al ejemplo de la secuencia de ADN. Supongamos que necesitamos dividir una secuencia dada usando los tripletas de genes (*codones*). La implementación mediante expresiones regulares para separar la siguiente secuencia usando las tripleras *GGG* y *GGA* sería:

In [60]:
adn = "AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGA" + \
"ACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCC" + \
"ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAG"

print(re.split('(?:GGG|GGA)', adn))


['AGCTTTTCATTCTGACTGCAAC', 'CAATATGTCTCTGTGT', 'TTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGC', 'CTGACGCGTACA', 'AACACAGAAAAAAG']


En este caso hemos al incorporar los meta-caracteres `?:` estamos indicando que vamos a buscar cualquiera de los grupos de  expresiones dentro de los paréntesis, pero que las subsecuencias que coincidan no serán retornadas después realizar la búsqueda. Esto permite que split solo retorne las secuencias producto de la división. Esto se conoce como *non-capturing version*.

Los conceptos vistos en este material corresponden a los usos básicos que pueden dar a las expresiones regulares. Les recomendamos revisar la documentación de Python para [expresiones regulares](https://docs.python.org/3/library/re.html) y [HOWTO](https://docs.python.org/3/howto/regex.html#regex-howto), para tener mayor detalle de cada método en el módulo `re` y conocer otros casos de uso. También pueden verificar las expresiones regular que escriban usando sitios como [http://pythex.org/](http://pythex.org/)